In [20]:
import sys
sys.path.append('../')

from interface.fixedleg import *
from interface.floatingleg import *
from datetime import date, timedelta
from models.hullwhitesimple import *
from engines.gaussian1d import *
from index.curverateindex import *

## Step1 - simulate future curves

In [9]:
t0_pillars = [0.25, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0]
t0_rates = [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
t0_rate_curve = RateCurveSimple(t0_pillars, t0_rates)

# create an instance of the hull-white model
mean_rev = 0.02
sigma = 0.045
hw = HullWhiteProcess(mean_rev, sigma, market_curve=t0_rate_curve)

# simulate short rate and discount curve 
last_grid_time = 20
time_steps = 50
hw_kernel = GaussianRateKernel1D(hw)
r_tensor, zb_tensor, grid = hw_kernel.rate_tensor(1000, last_grid_time, time_steps)

# Reshape the tensor and create a discount curve object
reshaped_tensor = tf.transpose(zb_tensor, perm=[1, 2, 0])  # Transpose the dimensions
reshaped_tensor = tf.reshape(reshaped_tensor, shape=(8, 1000, 49))
rate_curve_tensor = DiscountCurveSimple(t0_pillars, reshaped_tensor)

In [13]:
# whenever you request a discount factor for a certain date you get it for all the tensor
rate_curve_tensor.discount(3)

<tf.Tensor: shape=(1000, 49), dtype=float64, numpy=
array([[0.84465449, 0.81970654, 0.79203824, ..., 0.80458586, 0.80183014,
        0.75789296],
       [0.87331892, 0.82643089, 0.8171063 , ..., 1.06431006, 0.98748379,
        1.00417306],
       [0.89839965, 0.87187958, 0.88533145, ..., 0.75314001, 0.77446832,
        0.78471321],
       ...,
       [0.87335843, 0.87160062, 0.84270685, ..., 0.60892653, 0.60323493,
        0.61784466],
       [0.84567014, 0.85238993, 0.82659519, ..., 0.76451225, 0.71702223,
        0.69153658],
       [0.89744583, 0.86855019, 0.90422685, ..., 0.72903713, 0.74854627,
        0.71548407]])>

## Step2 - Fixed Legs

In [17]:
evaluation_date = date(2023, 6, 21)
start_date = date(2015,12,31)
fixed_schedule = [start_date]
fixed_schedule.extend([start_date + timedelta(365 *.5 *i) for i in range(1,51)])
fixed_rate = [0.04156]*(len(fixed_schedule)-1)
fixed_notional = [143857602,142099704,140995554,139343227,138170625,136487409,135100610,133235348,131633135,130088055,
                128443001,126527961,124478813,122046073,119540895,117334856,115325682,113682288,112374057,110400222,108996590,
                107427609,106277787,104934949,103826978,102318265,101011839,99241108,97427949,95062460,92643968,89488286,86306759,
                82654668,79233393,75249260,71491885,67239725,63147273,58562602,54101515,49235670,44484930,39276217,34122360,28587822,
                23186787,17327243,11581997,5627771]
# create the FixedRateLeg object
leg1 = FixedRateLeg(fixed_schedule, fixed_notional, fixed_rate)

# price is computed for all the input tensor
leg1.npv(rate_curve_tensor, evaluation_date)

<tf.Tensor: shape=(1000, 49), dtype=float64, numpy=
array([[41316926.22696852, 39278180.42472614, 37123254.56482752, ...,
        37235897.500861  , 37028732.62358788, 33944958.20895715],
       [43705209.8735273 , 39806603.9402761 , 39032394.27480633, ...,
        60216390.42280106, 52503562.93844229, 54092645.23169626],
       [45883158.8679911 , 43527270.26485293, 44623547.35531961, ...,
        33635718.96368721, 35087470.20264274, 35798487.40723414],
       ...,
       [43708575.30030467, 43503625.48725204, 41061371.83342092, ...,
        24968867.87188123, 24663750.67099166, 25442267.40160827],
       [41399745.44125623, 41899542.42874496, 39774973.64620239, ...,
        34407152.07846751, 31269664.0470104 , 29679052.20650442],
       [45798796.12433001, 43245730.09185938, 46278522.17623712, ...,
        32045220.09410423, 33322311.40680351, 31167393.27691526]])>

## Step3 - Floating Legs

In [23]:
floating_schedule = [start_date]
floating_schedule.extend([start_date + timedelta(365 *.5 *i) for i in range(1,51)])

floating_notional = [0,0,0,0,0,0,0,0,0,0,
    128443001,126527961,124478813,122046073,119540895,117334856,115325682,113682288,112374057,110400222,108996590,
    107427609,106277787,104934949,103826978,102318265,101011839,99241108,97427949,95062460,92643968,89488286,86306759,
    82654668,79233393,75249260,71491885,67239725,63147273,58562602,54101515,49235670,44484930,39276217,34122360,28587822,
    23186787,17327243,11581997,5627771]

index_6m = CurveRateIndex("EUR6M", "calendar", t0_rate_curve, "6M")
spread = [0.] *(len(floating_schedule)-1)
gearing = [1.] *(len(floating_schedule)-1)

float_leg = FloatingRateLeg(floating_schedule,
                            floating_notional,
                            gearing,
                            spread,
                            index_6m)
index_6m.add_fixing(date(2022,12,29), 0.03)

# price and fixings are computed for all the input tensor
npv_float = float_leg.npv(rate_curve_tensor, evaluation_date)



## under construction -- next steps

In [24]:
# scale the tensor approach for a large portfolio, divided by currency.
# modify the pricing function such that ref_date (for future valuation) is passed by the InterestCurve object